In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import torch
import pandas as pd
import tqdm
from torch.utils.data import Dataset, DataLoader
import glob
import dfs_code
from torch_geometric.data import InMemoryDataset, Data

In [6]:
n_splits = 10
use_Hs = False
path = "../../results/pubchem/noH"

In [10]:
i = 4
with open(path+"/%d/min_dfs_codes_split%d.json"%(i+1, i), 'r') as f:
    codes = json.load(f)

In [12]:
len(codes)

999262

In [7]:
codes_all = {}
for i in range(n_splits):
    with open(path+"/%d/min_dfs_codes_split%d.json"%(i+1, i), 'r') as f:
        codes = json.load(f)
        for key, val in tqdm.tqdm(codes.items()):
            codes_all[key] = val

FileNotFoundError: [Errno 2] No such file or directory: '../../results/pubchem/noH/1/min_dfs_codes_split0.json'

In [20]:
len(dataset)

59629

In [21]:
dataset[50000]

Data(edge_attr=[64, 4], edge_index=[2, 64], min_dfs_code=[32, 8], min_dfs_index=[29], name="CHEMBL1182990", smiles="CCCCN1CCN(C2=Nc3ccc(CC)cc3CC=C2c2ccccc2)CC1", x=[29, 39], z=[29])

In [4]:
path = "/home/chrisw/Documents/projects/2021/graph-transformer/results/chembl_noHs/amd-server/leq100_timeout60_sanity"

In [5]:
graph_pattern = "/home/chrisw/Documents/projects/2021/graph-transformer/datasets/ChEMBL/preprocessedNoHs_split%d.pt"

In [8]:
len(codes_all)/1920138

0.9937624274921907

In [9]:
valid_list = []
for d in codes_all.values():
    valid_list += [dfs_code.isValidDFSCode(d['min_dfs_code'])]

In [10]:
np.asarray(valid_list).sum() == len(valid_list)

True

In [11]:
len(valid_list)

1908161

In [12]:
ids = []
for i in range(64):
    with open(path+"/%d/metrics.json"%(i+1), 'r') as f:
        metrics = json.load(f)
        for key, value in metrics.items():
            if "time " in key:
                ids += [key[5:]]

In [13]:
len(ids)

1908161

In [14]:
sanity2 = np.asarray([id in codes_all for id in ids])

In [15]:
sanity2.sum() == len(sanity2)

True

In [16]:
df = pd.read_csv('../../datasets/ChEMBL/small_molecules.csv', delimiter=';')

/home/chrisw/.cache/pypoetry/virtualenvs/graph-transformer-9jPERXQ--py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
target_dir = '../../datasets/ChEMBL100_noH/'

In [18]:
import pickle

In [19]:
import tarfile
import os.path
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [20]:
chembl2smiles = {cid:smiles for cid, smiles in zip(df['ChEMBL ID'], df['Smiles'])}

In [21]:
for i in range(64):
    dataset = ChEMBL(graph_pattern%i)
    for d in tqdm.tqdm(dataset):
        if d.name in codes_all:
            code = codes_all[d.name]
            d_dict = {'x':d.x.numpy(),
                      'z':d.z.numpy(),
                      'edge_attr': d.edge_attr.numpy(),
                      'edge_index': d.edge_index.numpy(),
                      'name':d.name,
                      'min_dfs_code': np.asarray(code['min_dfs_code']),
                      'min_dfs_index': np.asarray(code['dfs_index']),
                      'smiles': chembl2smiles[d.name]}
            with open(target_dir+d.name+'.pkl', 'wb') as f:
                pickle.dump(d_dict, f)
    del dataset

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29916/29916 [00:06<00:00, 4346.52it/s]


for i in tqdm.tqdm(range(64)):
    dataset = ChEMBL(graph_pattern%i)
    for d in dataset:
        if d.name in codes_all:
            code = codes_all[d.name]
            d_ = Data(x=d.x, z=d.z, edge_attr=d.edge_attr, edge_index=d.edge_index, name=d.name, 
                      min_dfs_code=torch.tensor(code['min_dfs_code']), 
                      min_dfs_index=torch.tensor(code['dfs_index']),
                      smiles=df['Smiles'][df['ChEMBL ID'] == d.name].to_numpy()[0]) # note that 
                      # for some disconnected molecules i just used the largest connected component
            #torch.save(d_, target_dir+d.name+'.pt')
            #make_tarfile(target_dir+d.name+'.tar.gz', target_dir+d.name+'.pt')
            d_dict = {'x':d_.x.numpy(),
                      'z':d_.z.numpy(),
                      'edge_attr': d_.edge_attr.numpy(),
                      'edge_index': d_.edge_index.numpy(),
                      'name':d_.name,
                      'min_dfs_code': d_.min_dfs_code.numpy(),
                      'min_dfs_index': d_.min_dfs_index.numpy(),
                      'smiles': d_.smiles}
            with open(target_dir+d.name+'.pkl', 'wb') as f:
                pickle.dump(d_dict, f)
            
            #print(d_.x)
            #print(d_.z)
            #print(d_.edge_attr)
            #print(d_.edge_index)
            #print(d_.min_dfs_code)
            #print(d_.min_dfs_index)
            #print(d_.smiles)


In [ ]:
make_tarfile(target_dir[:-1]+'.tar.gz', target_dir[:-1])